Ensuring we use apple m1 metal for processing power

In [4]:
import torch
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


Data train split minimum 20 faces

In [13]:
import os
import shutil
import numpy as np
from sklearn.model_selection import train_test_split
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense, Flatten
from keras.preprocessing.image import ImageDataGenerator

source_dir = 'photos/lfw_funneled'
categories = os.listdir(source_dir)

# directories for training and testing sets
train_dir = 'photos/train_set'
test_dir = 'photos/test_set'

# training and testing directories
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# For each category(person), split images into training and testing sets
for category in categories:
    # Error checking skips if this is a file, not a directory
    if not os.path.isdir(os.path.join(source_dir, category)):
        continue
    
    image_list = os.listdir(os.path.join(source_dir, category))
    
    # We only only need those with minimum 20 faces
    if len(image_list) < 20:
        # print(f'Skipping {category}, only {len(image_list)} images found')
        continue
    
    # We should shuffle the image list
    np.random.shuffle(image_list)

    # Splitting data into training and testing sets
    train_list, test_list = train_test_split(image_list, test_size=0.2)

    # category directories in training and testing directories
    os.makedirs(os.path.join(train_dir, category), exist_ok=True)
    os.makedirs(os.path.join(test_dir, category), exist_ok=True)

    # Copy images to appropriate directories
    for image in train_list:
        shutil.copy(os.path.join(source_dir, category, image), os.path.join(train_dir, category))
    for image in test_list:
        shutil.copy(os.path.join(source_dir, category, image), os.path.join(test_dir, category))

    # Copy images to appropriate directories
    for image in train_list:
        shutil.copy(os.path.join(source_dir, category, image), os.path.join(train_dir, category))
    for image in test_list:
        shutil.copy(os.path.join(source_dir, category, image), os.path.join(test_dir, category))


In [6]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense, Flatten
from keras.preprocessing.image import ImageDataGenerator


# Load pre-trained the VGG16 model on imagenet
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

# Some manipulation
x = Flatten()(base_model.output)

x = Dense(1024, activation='relu')(x)

# This parameter depends on number of our classes/people, either we know it or error will tell us...
output = Dense(62, activation='softmax')(x)

# defining the model
model = Model(inputs=base_model.input, outputs=output)

for layer in base_model.layers:
    layer.trainable = False

# Compilation of the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory('photos/train_set', target_size=(64, 64), batch_size=32, class_mode='categorical')
test_set = test_datagen.flow_from_directory('photos/test_set', target_size=(64, 64), batch_size=32, class_mode='categorical')

# Train the model
model.fit(training_set, epochs=15, validation_data=test_set)



Found 2892 images belonging to 62 classes.
Found 1131 images belonging to 62 classes.
Epoch 1/15


2023-05-18 18:38:24.666384: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


91/91 [==============================] - 28s 305ms/step - loss: 3.4927 - accuracy: 0.2044 - val_loss: 2.9054 - val_accuracy: 0.2405
Epoch 2/15
91/91 [==============================] - 27s 302ms/step - loss: 2.8059 - accuracy: 0.3050 - val_loss: 2.3596 - val_accuracy: 0.3599
Epoch 3/15
91/91 [==============================] - 28s 306ms/step - loss: 2.4242 - accuracy: 0.3814 - val_loss: 2.1681 - val_accuracy: 0.4076
Epoch 4/15
91/91 [==============================] - 29s 321ms/step - loss: 2.1813 - accuracy: 0.4108 - val_loss: 1.9010 - val_accuracy: 0.4996
Epoch 5/15
91/91 [==============================] - 28s 303ms/step - loss: 1.9578 - accuracy: 0.4703 - val_loss: 1.6391 - val_accuracy: 0.5632
Epoch 6/15
91/91 [==============================] - 22s 246ms/step - loss: 1.8127 - accuracy: 0.4965 - val_loss: 1.5982 - val_accuracy: 0.5836
Epoch 7/15
91/91 [==============================] - 22s 246ms/step - loss: 1.6821 - accuracy: 0.5377 - val_loss: 1.4274 - val_accuracy: 0.6048
Epoch 8/15

In [8]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense, Flatten
from keras.preprocessing.image import ImageDataGenerator

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

x = Flatten()(base_model.output)

x = Dense(1024, activation='relu')(x)

output = Dense(62, activation='softmax')(x) 

model = Model(inputs=base_model.input, outputs=output)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory('photos/train_set', target_size=(224, 224), batch_size=32, class_mode='categorical')
test_set = test_datagen.flow_from_directory('photos/test_set', target_size=(224, 224), batch_size=32, class_mode='categorical')

model.fit(training_set, epochs=15, validation_data=test_set)


Found 2892 images belonging to 62 classes.
Found 1131 images belonging to 62 classes.
Epoch 1/15
91/91 [==============================] - 246s 3s/step - loss: 4.7528 - accuracy: 0.2441 - val_loss: 2.4915 - val_accuracy: 0.3722
Epoch 2/15
91/91 [==============================] - 251s 3s/step - loss: 2.0666 - accuracy: 0.5048 - val_loss: 1.4538 - val_accuracy: 0.6569
Epoch 3/15
91/91 [==============================] - 250s 3s/step - loss: 1.3244 - accuracy: 0.6746 - val_loss: 1.0593 - val_accuracy: 0.7321
Epoch 4/15
91/91 [==============================] - 245s 3s/step - loss: 0.9026 - accuracy: 0.7766 - val_loss: 0.7155 - val_accuracy: 0.8258
Epoch 5/15
91/91 [==============================] - 244s 3s/step - loss: 0.6061 - accuracy: 0.8562 - val_loss: 0.4220 - val_accuracy: 0.8930
Epoch 6/15
91/91 [==============================] - 245s 3s/step - loss: 0.4031 - accuracy: 0.9160 - val_loss: 0.2789 - val_accuracy: 0.9355
Epoch 7/15
91/91 [==============================] - 239s 3s/step - l

In [9]:
model.save('98%.h5')

In [10]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

# Load the saved model
model = load_model('98%.h5')



# retrieve  the labels map from the training set
label_map = (training_set.class_indices)
label_map = dict((v,k) for k,v in label_map.items())

def classify_image(image_path, model, label_map):
    # Load the images
    img = load_img(image_path, target_size=(224, 224))

    # Convert the images to array
    img_array = img_to_array(img)

    # dimensions must  match the shape that model expects
    img_batch = np.expand_dims(img_array, axis=0)

    # normalisatiom of image
    img_preprocessed = img_batch / 255.

    # get the class of the image using our pretrianed model, verbose = reduces print size
    predictions = model.predict(img_preprocessed, verbose = 0)

    # get the class index that has the highest probability
    class_index = np.argmax(predictions[0])

    # finaly we get  the label of class
    class_label = label_map[class_index]

    return class_label




In [11]:
image_path = 'photos/train_set/Vladimir_Putin/Vladimir_Putin_0005.jpg'
class_label = classify_image(image_path, model, label_map)
print("This image is likely to be of class:", class_label)

This image is likely to be of class: Vladimir_Putin


Above works fine for singular images but we have thousands of images to test, lets make loop

In [12]:


import os

def classify_images_in_directory(directory_path, model, label_map):
    results = {}

    for subdir, dirs, files in os.walk(directory_path):
        for file in files:
            image_path = os.path.join(subdir, file)
            class_label = classify_image(image_path, model, label_map)
            file_name = os.path.splitext(os.path.basename(image_path))[0] 
            results[file_name] = class_label

    return results

directory_path = 'photos/test_set'
results = classify_images_in_directory(directory_path, model, label_map)

for image_name, class_label in results.items():
    print(f"\nThe image {image_name} is likely to be of class: {class_label}")



The image Jennifer_Aniston_0005 is likely to be of class: Jennifer_Aniston

The image Jennifer_Aniston_0011 is likely to be of class: Jennifer_Aniston

The image Jennifer_Aniston_0007 is likely to be of class: Jennifer_Aniston

The image Jennifer_Aniston_0012 is likely to be of class: Jennifer_Aniston

The image Jennifer_Aniston_0002 is likely to be of class: Jennifer_Aniston

The image Jennifer_Aniston_0016 is likely to be of class: Jennifer_Aniston

The image Jennifer_Aniston_0003 is likely to be of class: Jennifer_Aniston

The image Jennifer_Aniston_0014 is likely to be of class: Jennifer_Aniston

The image Jennifer_Aniston_0019 is likely to be of class: Jennifer_Aniston

The image Tom_Daschle_0003 is likely to be of class: Tom_Daschle

The image Tom_Daschle_0017 is likely to be of class: Tom_Daschle

The image Tom_Daschle_0006 is likely to be of class: Tom_Daschle

The image Tom_Daschle_0012 is likely to be of class: Tom_Daschle

The image Tom_Daschle_0020 is likely to be of class